In [ ]:
# Install required packages
!pip install pandas deep-translator langdetect nltk Sastrawi retry -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 4.3 MB/s eta 0:00:00


In [ ]:
# Uninstall NLTK versi saat ini
!pip uninstall nltk -y -q

# Install NLTK versi 3.8.1 (versi stabil yang diketahui berfungsi)
!pip install nltk==3.8.1 -q

import nltk

# Download NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

# Lanjutkan dengan kode augmentasi lainnya

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 14.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
textblob 0.19.0 requires nltk>=3.9, but you have nltk 3.8.1 which is incompatible.


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
import re
import logging
from deep_translator import GoogleTranslator
from langdetect import detect, LangDetectException
import random
import time
from retry import retry
from nltk.tokenize import word_tokenize
import nltk
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from multiprocessing import Pool
import numpy as np

# Download NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')

# Inisialisasi logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# Load file CSV hasil preprocessing
try:
    df = pd.read_csv('ikn_news_preprocessed_before_augmentation.csv')
    logger.info(f"Berhasil memuat file ikn_news_preprocessed_before_augmentation.csv dengan {len(df)} baris")
except FileNotFoundError:
    logger.error("File ikn_news_preprocessed_before_augmentation.csv tidak ditemukan.")
    raise

# Validasi kolom
required_columns = ['judul', 'cleaned_content', 'label']
if not all(col in df.columns for col in required_columns):
    missing_cols = [col for col in required_columns if col not in df.columns]
    logger.error(f"Kolom yang hilang: {missing_cols}")
    raise ValueError(f"DataFrame tidak memiliki kolom yang diperlukan: {missing_cols}")

# Definisikan kata kunci IKN untuk validasi relevansi
ikn_keywords = [
    'ikn', 'nusantara', 'ibu kota negara', 'pemindahan ibu kota', 'penajam paser utara',
    'kalimantan timur', 'kaltim', 'pembangunan ikn', 'otorita ikn', 'infrastruktur ikn',
    'investasi ikn', 'jokowi', 'istana negara', 'kementerian', 'proyek ikn'
]

# Inisialisasi translator dengan caching
translation_cache = {}
translator = GoogleTranslator(source='auto', target='en')
translator_id = GoogleTranslator(source='en', target='id')

# Kamus sinonim yang lebih kaya
synonym_dict = {
    'tidak': ['bukan', 'tak', 'jangan'],
    'pembangunan': ['konstruksi', 'pengembangan', 'pendirian'],
    'proyek': ['rencana', 'pekerjaan', 'program'],
    'investasi': ['penanaman modal', 'pendanaan', 'modal'],
    'infrastruktur': ['sarana', 'prasarana', 'fasilitas'],
    'anggaran': ['dana', 'biaya', 'belanja'],
    'pemerintah': ['penguasa', 'badan', 'otoritas'],
    'kota': ['perkotaan', 'ibu kota', 'wilayah'],
    'jokowi': ['presiden', 'joko widodo'],
    'kaltim': ['kalimantan timur'],
    'nusantara': ['ibu kota nusantara', 'ikn'],
    'pemindahan': ['relokasi', 'perpindahan']
}

# Inisialisasi stemmer untuk normalisasi kata
stemmer_factory = StemmerFactory()
stemmer = stemmer_factory.create_stemmer()

# Fungsi untuk penggantian sinonim
def synonym_replacement(text, n=3):
    try:
        words = word_tokenize(text)
        new_words = words.copy()
        indices = list(range(len(words)))
        random.shuffle(indices)
        replaced = 0
        for i in indices:
            word = words[i].lower()
            if word in synonym_dict:
                synonyms = [syn for syn in synonym_dict[word] if syn != word]
                if synonyms:
                    new_words[i] = random.choice(synonyms)
                    replaced += 1
            if replaced >= n:
                break
        return " ".join(new_words)
    except Exception as e:
        logger.error(f"Error in synonym_replacement: {e}")
        return text

# Fungsi untuk paraphrasing sederhana (mengubah urutan kata penting)
def simple_paraphrase(text):
    try:
        words = word_tokenize(text)
        if len(words) < 5:
            return text
        key_positions = [i for i, word in enumerate(words) if word.lower() in ikn_keywords]
        if len(key_positions) < 2:
            return text
        # Swap dua kata kunci acak
        pos1, pos2 = random.sample(key_positions, 2)
        words[pos1], words[pos2] = words[pos2], words[pos1]
        return " ".join(words)
    except Exception as e:
        logger.error(f"Error in simple_paraphrase: {e}")
        return text

# Fungsi untuk memotong teks panjang
def truncate_text(text, max_length=5000):
    if len(text) > max_length:
        logger.warning(f"Teks terlalu panjang ({len(text)} karakter), dipotong menjadi {max_length} karakter.")
        return text[:max_length]
    return text

# Fungsi untuk back-translation dengan retry
@retry(tries=3, delay=1, backoff=2)
def back_translate(text, intermediate_lang='en'):
    try:
        text = truncate_text(text)
        if text in translation_cache:
            return translation_cache[text]
        translated = GoogleTranslator(source='id', target=intermediate_lang).translate(text)
        time.sleep(0.5)
        back_translated = GoogleTranslator(source=intermediate_lang, target='id').translate(translated)
        time.sleep(0.5)
        translation_cache[text] = back_translated
        return back_translated
    except Exception as e:
        logger.error(f"Error in back-translation: {e}")
        return text

# Fungsi untuk validasi teks hasil augmentasi
def validate_augmented_text(text):
    try:
        if len(text.split()) < 5:
            logger.warning(f"Teks terlalu pendek: {text[:50]}...")
            return False
        if detect(text) != 'id':
            logger.warning(f"Teks tidak terdeteksi sebagai bahasa Indonesia: {text[:50]}...")
            return False
        keyword_count = sum(1 for keyword in ikn_keywords if keyword.lower() in text.lower())
        if keyword_count < 2:
            logger.warning(f"Teks kurang relevan dengan IKN (hanya {keyword_count} kata kunci): {text[:50]}...")
            return False
        return True
    except LangDetectException:
        logger.warning(f"Deteksi bahasa gagal: {text[:50]}...")
        return False
    except Exception as e:
        logger.error(f"Error in validate_augmented_text: {e}")
        return False

# Fungsi untuk membersihkan teks hasil augmentasi
def clean_augmented_text(text):
    try:
        text = re.sub(r'\s+', ' ', text).strip()
        text = re.sub(r'[^\w\s.,!?éê]', '', text)
        # Normalisasi kata dengan stemming
        words = word_tokenize(text)
        stemmed_words = [stemmer.stem(word) if word.lower() not in ikn_keywords else word for word in words]
        return " ".join(stemmed_words)
    except Exception as e:
        logger.error(f"Error in clean_augmented_text: {e}")
        return text

# Fungsi augmentasi untuk satu baris
def augment_row(row, intermediate_langs):
    try:
        original_text = row['cleaned_content']
        method = random.choice(['back_translate', 'synonym', 'paraphrase'])

        if method == 'back_translate':
            intermediate_lang = random.choice(intermediate_langs)
            augmented_text = back_translate(original_text, intermediate_lang=intermediate_lang)
        elif method == 'synonym':
            augmented_text = synonym_replacement(original_text, n=3)
        else:
            augmented_text = simple_paraphrase(original_text)

        augmented_text = clean_augmented_text(augmented_text)
        if validate_augmented_text(augmented_text):
            row['cleaned_content'] = augmented_text
            row['is_augmented'] = True
            return row
        return None
    except Exception as e:
        logger.error(f"Error in augment_row: {e}")
        return None

# Parameter augmentasi
TARGET_PER_LABEL = 100
INTERMEDIATE_LANGS = ['en', 'fr', 'de']
dfs = []

logger.info("Mulai proses augmentasi per label...")

for label in df['label'].unique():
    subset = df[df['label'] == label].copy()
    count = len(subset)
    logger.info(f"Label '{label}' memiliki {count} data")

    if count >= TARGET_PER_LABEL:
        sampled = subset.sample(n=TARGET_PER_LABEL, random_state=42).copy()
        sampled['is_augmented'] = False
        dfs.append(sampled)
    else:
        subset['is_augmented'] = False
        dfs.append(subset)

        # Hitung jumlah data yang perlu diaugmentasi
        n_to_augment = TARGET_PER_LABEL - count
        logger.info(f"Label '{label}': Perlu augmentasi {n_to_augment} data")

        # Proses augmentasi menggunakan multiprocessing
        rows_to_augment = [subset.iloc[random.randint(0, count-1)].copy() for _ in range(n_to_augment * 2)]
        with Pool(processes=4) as pool:
            results = pool.starmap(augment_row, [(row, INTERMEDIATE_LANGS) for row in rows_to_augment])

        # Filter hasil yang valid
        augmented_rows = [row for row in results if row is not None][:n_to_augment]

        # Jika masih kurang, gunakan data asli sebagai fallback
        if len(augmented_rows) < n_to_augment:
            logger.warning(f"Label '{label}': Hanya menghasilkan {len(augmented_rows)} dari {n_to_augment} data augmentasi. Mengisi sisanya dengan data asli.")
            remaining = n_to_augment - len(augmented_rows)
            for i in range(remaining):
                base_row = subset.iloc[random.randint(0, count-1)].copy()
                base_row['is_augmented'] = False
                augmented_rows.append(base_row)

        df_aug = pd.DataFrame(augmented_rows)
        dfs.append(df_aug)
        logger.info(f"Label '{label}': Total data setelah augmentasi: {len(augmented_rows) + count}")

# Gabungkan semua hasil
df_final = pd.concat(dfs, ignore_index=True)

# Simpan ke CSV dan Excel
df_final.to_csv('ikn_news_augmented_full_pipeline.csv', index=False)
df_final.to_excel('ikn_news_augmented_full_pipeline.xlsx', index=False)
logger.info("Proses augmentasi selesai, data disimpan ke ikn_news_augmented_full_pipeline.csv dan .xlsx")

# Tampilkan ringkasan
logger.info(f"Total data akhir: {len(df_final)}")
logger.info(f"Distribusi label:\n{df_final['label'].value_counts().to_string()}")
logger.info("Sampel 5 baris pertama setelah augmentasi:")
logger.info(df_final[['judul', 'cleaned_content', 'label', 'is_augmented']].head().to_string())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
ERROR:__main__:Error in back-translation: catatan artikel ini merupakan opini pribadi penulis dan tidak mencerminkan pandangan redaksi bayangkan sebuah kota dengan berbagai keajaibannya. kereta tanpa rel atau autonomousrail rapid transit art, taksi terbang atau advanced air mobility aam, tiang cerdas multifungsi smart pole, serta teknologi futuristik lainnya menghiasi lingkungan kota. itulah gambaran sekilas __ikn__ __ikn__, ibu kota negara baru indonesia di __kaltim__. proyek yang terkesan ambisius ini telah menimbulkan banyak diskursus di antara kita. banyak yang tak mendugabahkan tak percayaakankah proyek ini nyata status quo proyek ibu kota nusantarasaat ini, realita pembangunan nasional yang terkesan jawasentris menjadi masalah sekaligus tantangan tersend